### Import data (Redit Pkl file)

In [ ]:
import pickle
import numpy as np
import pandas as pd

In [ ]:
# reddit = praw.Reddit(
#     client_id="",
#     client_secret="",
#     user_agent="",
#     username='',
#     password='')

# #please read this website to get all the information needed: https://towardsdatascience.com/scraping-reddit-data-1c0af3040768
# print(reddit.user.me())

# keyword = 'apple' # i still need to build a for loop so we can do this scraping for all the companies
# num_days = 30
# time_filter = int((datetime.datetime.utcnow() - datetime.timedelta(days=num_days)).timestamp())
# subreddit = reddit.subreddit(keyword)
# hot_posts = subreddit.hot(limit=1000)
# new_posts = subreddit.new(limit=1000)

# # Search for posts that contain the keyword within the given time frame
# search_results = hot_posts
# post_dict = {}

# # Iterate through the search results
# for post in search_results:
#     # Check if the post is within the date range
#     if post.created_utc > time_filter:
#         # Create a dictionary for the post and add it to the post_dict
#         post_info = {
#             'date': datetime.datetime.fromtimestamp(post.created_utc).strftime('%y/%m/%d'),
#             'company_name': keyword,
#             'upvotes': post.score,
#             'title':post.title,
#             'sentence': {}
#         }
#         post_dict[post.id] = post_info
        
#         # Get the text of the post
#         post_text = post.selftext
        
#         # Split the text into sentences and add to sentence_dict
#         sentence_dict = {}
#         sentences = re.split('[.!?]', post_text)
#         for i, sentence in enumerate(sentences):
#             clean_sentence = re.sub('[^0-9a-zA-Z]+', ' ', sentence).strip()
#             if clean_sentence:
#                 sentence_dict[i+1] = clean_sentence
#         post_dict[post.id]['sentence'] = sentence_dict
        
#         # Loop through the comments and add to sentence_dict
# #         post.comments.replace_more(limit=None)
# #         for comment in post.comments:
# #             comment_text = comment.body
# #             comment_sentences = re.split('[.!?]', comment_text)
# #             for i, sentence in enumerate(comment_sentences):
# #                 clean_sentence = re.sub('[^0-9a-zA-Z]+', ' ', sentence).strip()
# #                 if clean_sentence:
# #                     sentence_dict[i+1] = clean_sentence
        

# # Print the post_dict
# print(post_dict)

# file_name = f'post_dict_{num_days}d_{keyword}.pkl'

# with open(file_name, 'wb') as f:
#     pickle.dump(post_dict, f)

In [ ]:
with open('post_dict_30d_apple.pkl', 'rb') as f:
    pkl_apple_30d = pickle.load(f)

In [ ]:
len(pkl_apple_30d.keys())

369

In [ ]:
# dict_apple_30d = {'date': np.nan, 'company_name': np.nan, 'sentence': np.nan, 'upvotes': np.nan, 'title': np.nan}

### Refine the scrapped data - Drop the datapoint with no sentence

In [ ]:
filtered = []

for i in pkl_apple_30d:
    if pkl_apple_30d[i]['sentence']:
        filtered.append(pkl_apple_30d[i])

df_apple_30d = pd.DataFrame(filtered)

In [ ]:
df_apple_30d['sentence_list'] =  df_apple_30d['sentence'].apply(lambda x: list(x.values()))

In [ ]:
df_apple_30d['date_format'] = df_apple_30d['date'].apply(lambda x: pd.to_datetime(x, format = '%y/%m/%d'))

In [ ]:
df_apple_30d.head()

,date,company_name,upvotes,title,sentence,sentence_list,date_format
0,23/03/24,apple,3,"Daily Advice Thread - March 24, 2023",{1: 'Welcome to the Daily Advice Thread for r ...,[Welcome to the Daily Advice Thread for r Appl...,2023-03-24
1,23/03/23,apple,25,Apple should bring iPhone as a webcam to Apple TV,"{1: 'The title says it all', 2: 'I hope apple ...","[The title says it all, I hope apple is bringi...",2023-03-23
2,23/03/23,apple,8,Anyone else have lots of connectivity issues w...,{1: 'Me and my partner have the newest Apple W...,[Me and my partner have the newest Apple Watch...,2023-03-23
3,23/03/23,apple,9,Should Apple enable 1440p 60fps video recordin...,{1: 'I like to use default camera app to recor...,[I like to use default camera app to record vi...,2023-03-23
4,23/03/22,apple,92,The Health App Needs Fixing,{1: 'I can t believe Apple still hasn t fixed ...,[I can t believe Apple still hasn t fixed the ...,2023-03-22


In [ ]:
apple_prep = df_apple_30d[['date_format', 'sentence_list']]

In [ ]:
apple_prep.fillna(0)

,date_format,sentence_list
0,2023-03-24,[Welcome to the Daily Advice Thread for r Appl...
1,2023-03-23,"[The title says it all, I hope apple is bringi..."
2,2023-03-23,[Me and my partner have the newest Apple Watch...
3,2023-03-23,[I like to use default camera app to record vi...
4,2023-03-22,[I can t believe Apple still hasn t fixed the ...
...,...,...
160,2023-02-23,[It s a bug that will play audio from a source...
161,2023-02-23,[I have owned the AirPods Pro for a while and ...
162,2023-02-23,[Welcome to the Daily Advice Thread for r Appl...
163,2023-02-23,[I know a lot of people with iPads with M1 and...


In [ ]:
apple_prep.dtypes

date_format      datetime64[ns]
sentence_list            object
dtype: object

### 2023-02-22 ~ 2023-03-24

In [ ]:
date_list = list(set(apple_prep['date_format']))
date_list.sort()

In [ ]:
d = {'date_format': [], 'sentence_list': []}
apple_df = pd.DataFrame(data = d)

In [ ]:
apple_df

,date_format,sentence_list


In [ ]:
apple_df['date_format'] = date_list

In [ ]:
for i_df, v_df in enumerate(apple_df['date_format']):
    for i_prep, v_prep in enumerate(apple_prep['date_format']):
        if apple_df['date_format'][i_df] == apple_prep['date_format'][i_prep]:
            append_list = []
            append_list += apple_prep['sentence_list'][i_prep]
            apple_df['sentence_list'][i_df] = append_list


c:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\Anaconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [ ]:
apple_df.head()

,date_format,sentence_list
0,2023-02-22,[For Coding Ok after a long research I ve been...
1,2023-02-23,[I know a lot of people with iPads with M1 and...
2,2023-02-24,[Edit Read my update below before believing th...
3,2023-02-25,[Hi I see so many YouTubers that just keep on ...
4,2023-02-26,[Show stickies on iPhone iPad home screen Mac ...


### BERT Model for Sentiment Scoring

In [ ]:
# pip install tqdm==4.42.1

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
def sentiment_score_calculator(sentence_list):
    scores = []
    for sentence in sentence_list:

        # Tokenize the input sentence / 문장을 BERT 모델의 입력 형식에 맞게 변환
        input_ids = torch.tensor(tokenizer.encode(sentence)).unsqueeze(0)

        # Predict the sentiment score / 모델에 입력하여 sentiment score 예측
        outputs = model(input_ids)
        logits = outputs[0]

        # Convert to probability score / softmax 함수를 사용하여 확률값으로 변환
        scores.append(torch.softmax(logits, dim=1).tolist()[0][1])
    return sum(scores) / len(scores)

In [ ]:
apple_df['sentiment_score'] = apple_df['sentence_list'].apply(lambda x: sentiment_score_calculator(x))

### Daily sentiment score for Apple (from Reddit) is calculated
* Score can range between **0 (Negative)** and **1 (Positive)**.


In [ ]:
apple_df

,date_format,sentence_list,sentiment_score
0,2023-02-22,[For Coding Ok after a long research I ve been...,0.551417
1,2023-02-23,[I know a lot of people with iPads with M1 and...,0.576060
2,2023-02-24,[Edit Read my update below before believing th...,0.557167
3,2023-02-25,[Hi I see so many YouTubers that just keep on ...,0.547956
4,2023-02-26,[Show stickies on iPhone iPad home screen Mac ...,0.537351
5,2023-02-27,[Let s face it the iOS weather app has always ...,0.546029
6,2023-02-28,[Welcome to the Daily Advice Thread for r Appl...,0.548673
7,2023-03-01,[Welcome to the Daily Advice Thread for r Appl...,0.548673
8,2023-03-02,[I m massively annoyed by the lack of accounta...,0.542509
9,2023-03-03,[Welcome to the Daily Advice Thread for r Appl...,0.548673


### Import Historical Apple Stock Data

In [ ]:
apple_stock_raw = pd.read_csv('AAPL.csv')

In [ ]:
apple_stock_df = apple_stock_raw[['Date', 'Adj Close']]
apple_stock_df['Date'] = pd.to_datetime(apple_stock_df['Date'])
apple_stock_df.head()

c:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Date,Adj Close
0,2022-02-22,163.330612
1,2022-02-23,159.106186
2,2022-02-24,161.760117
3,2022-02-25,163.857407
4,2022-02-28,164.125763


In [ ]:
apple_merged = pd.merge(apple_df, apple_stock_df, left_on='date_format', right_on='Date', how = 'left')

In [ ]:
apple_merged

,date_format,sentence_list,sentiment_score,Date,Adj Close
0,2023-02-22,[For Coding Ok after a long research I ve been...,0.551417,2023-02-22,148.910004
1,2023-02-23,[I know a lot of people with iPads with M1 and...,0.576060,2023-02-23,149.399994
2,2023-02-24,[Edit Read my update below before believing th...,0.557167,2023-02-24,146.710007
3,2023-02-25,[Hi I see so many YouTubers that just keep on ...,0.547956,NaT,NaN
4,2023-02-26,[Show stickies on iPhone iPad home screen Mac ...,0.537351,NaT,NaN
5,2023-02-27,[Let s face it the iOS weather app has always ...,0.546029,2023-02-27,147.919998
6,2023-02-28,[Welcome to the Daily Advice Thread for r Appl...,0.548673,2023-02-28,147.410004
7,2023-03-01,[Welcome to the Daily Advice Thread for r Appl...,0.548673,2023-03-01,145.309998
8,2023-03-02,[I m massively annoyed by the lack of accounta...,0.542509,2023-03-02,145.910004
9,2023-03-03,[Welcome to the Daily Advice Thread for r Appl...,0.548673,2023-03-03,151.029999


### LSTM Model for Stock Price Prediction

In [ ]:
# pip install tensorflow

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
from tensorflow.keras.layers import LSTM, Dense, Input, Concatenate
from tensorflow.keras.models import Model

In [ ]:
apple_input = apple_merged.copy()
apple_input.dropna(inplace=True)
apple_input.drop(['date_format', 'sentence_list'], axis=1, inplace=True)

In [ ]:
apple_input

,sentiment_score,Date,Adj Close
0,0.551417,2023-02-22,148.910004
1,0.576060,2023-02-23,149.399994
2,0.557167,2023-02-24,146.710007
5,0.546029,2023-02-27,147.919998
6,0.548673,2023-02-28,147.410004
7,0.548673,2023-03-01,145.309998
8,0.542509,2023-03-02,145.910004
9,0.548673,2023-03-03,151.029999
12,0.548673,2023-03-06,153.830002
13,0.531726,2023-03-07,151.600006


In [ ]:
# Split data into train and test sets
train_data = apple_input[apple_input['Date'] < '2023-03-10']
test_data = apple_input[apple_input['Date'] >= '2023-03-10']

In [ ]:
# Normalize historical prices and sentiment scores
train_sentiment_score = (train_data['sentiment_score'] - train_data['sentiment_score'].mean()) / train_data['sentiment_score'].std()
train_price = (train_data['sentiment_score'] - train_data['sentiment_score'].mean()) / train_data['sentiment_score'].std()

test_sentiment_score = (test_data['sentiment_score'] - test_data['sentiment_score'].mean()) / test_data['sentiment_score'].std()
test_price = (test_data['sentiment_score'] - test_data['sentiment_score'].mean()) / test_data['sentiment_score'].std()

In [ ]:
train_sentiment_score

0     0.008865
1     2.060226
2     0.487563
5    -0.439638
6    -0.219490
7    -0.219490
8    -0.732629
9    -0.219490
12   -0.219490
13   -1.630223
14   -0.511848
15    1.635644
Name: sentiment_score, dtype: float64

In [ ]:
train_price


0     0.008865
1     2.060226
2     0.487563
5    -0.439638
6    -0.219490
7    -0.219490
8    -0.732629
9    -0.219490
12   -0.219490
13   -1.630223
14   -0.511848
15    1.635644
Name: sentiment_score, dtype: float64

In [ ]:
# Define input shapes
historical_price_input = Input(shape=(10, 1))
sentiment_score_input = Input(shape=(1,))

In [ ]:
# Define LSTM layers
lstm_layer = LSTM(64)(historical_price_input)

In [ ]:
# Concatenate LSTM output and sentiment score input
concatenated_inputs = Concatenate()([lstm_layer, sentiment_score_input])

In [ ]:
# Define output layer
output_layer = Dense(1, activation='linear')(concatenated_inputs)

In [ ]:
# Define the model
model = Model(inputs=[historical_price_input, sentiment_score_input], outputs=output_layer)

In [ ]:
# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
# normalize Adj Close and sentiment_score columns
# scaler = MinMaxScaler()
# apple_merged[['Adj Close', 'sentiment_score']] = scaler.fit_transform(apple_merged[['Adj Close', 'sentiment_score']])

In [ ]:
# Convert to numpy arrays
# sentiment_scores = apple_merged['sentiment_score'].values.reshape(-1, 1)
# adj_close = apple_merged['Adj Close'].values.reshape(-1, 1)

In [ ]:
# Scale the data
scaler = MinMaxScaler(feature_range=(0, 1))

sentiment_scores_scaled = scaler.fit_transform(sentiment_scores)
adj_close_scaled = scaler.fit_transform(adj_close)

# sentiment_scores_scaled = sentiment_scores_scaled.reshape(sentiment_scores_scaled.shape[0], sentiment_scores_scaled.shape[1], 1)
# adj_close_scaled = adj_close_scaled.reshape(adj_close_scaled.shape[0], adj_close_scaled.shape[1], 1)

# sentiment_scores_scaled = scaler.transform(apple_merged['sentiment_score'].values.reshape(-1,1))
# adj_close_scaled = scaler.transform(apple_merged['Adj Close'].values.reshape(-1,1))

In [ ]:
# Define the training data
train_X = []
train_y = []
for i in range(10, len(sentiment_scores)):
    train_X.append(sentiment_scores_scaled[i-10:i, 0])
    train_y.append(adj_close_scaled[i, 0])
train_X, train_y = np.array(train_X), np.array(train_y)

# Reshape the training data for LSTM
train_X = np.reshape(train_X, (train_X.shape[0], train_X.shape[1], 1))


In [ ]:
# Define the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(train_X.shape[1], 1)))
model.add(LSTM(units=50))
model.add(Dense(units=1))

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
# Train the model
model.fit(train_X, train_y, epochs=50, batch_size=32)

In [ ]:
# Define the test data
test_X = []
for i in range(len(sentiment_scores)-10, len(sentiment_scores)):
    test_X.append(sentiment_scores_scaled[i-10:i, 0])
test_X = np.array(test_X)

In [ ]:
# Reshape the test data for LSTM
test_X = np.reshape(test_X, (test_X.shape[0], test_X.shape[1], 1))

In [ ]:
# Predict the stock prices for the given dates
predicted_prices = []
for i in range(5):
    predicted_price = model.predict(np.array([sentiment_scores_scaled[-10-i:-i, 0]]))
    predicted_price = scaler.inverse_transform(predicted_price)
    predicted_prices.append(predicted_price[0][0])

In [ ]:
i = 0
np.array([sentiment_scores_scaled[-10-i:-i, 0]])

In [ ]:
# Predict the stock prices for the given dates
predicted_prices = []
for i in range(5):
    predicted_price = model.predict(np.array([sentiment_scores_scaled[-10-i:-i, 0]]))
    predicted_price = scaler.inverse_transform(predicted_price)
    predicted_prices.append(predicted_price[0][0])

# Print the predicted prices
dates = ['2023-03-04', '2023-03-05', '2023-03-06', '2023-03-07', '2023-03-08']
for i in range(5):
    print("Predicted stock price for {}: ${:.2f}".format(dates[i], predicted_prices[4-i]))


In [ ]:
# 데이터 로드
df = pd.read_csv('stock_data.csv')

# 입력 데이터와 출력 데이터 생성
X = []
y = []
for i in range(60, len(df)):
    X.append([df['date'][i-60:i], df['stock_price'][i-60:i], df['sentiment_score'][i]])
    y.append(df['stock_price'][i])

# 학습 데이터와 검증 데이터 분리
train_size = int(len(X) * 0.7)
train_X, train_y = X[:train_size], y[:train_size]
test_X, test_y = X[train_size:], y[train_size:]

# LSTM 모델 정의
model = Sequential()
model.add(LSTM(50, input_shape=(60, 2), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50))
model.add(Dense(1))

# 모델 학습 설정
model.compile(loss='mean_squared_error', optimizer='adam')

# 모델 학습
model.fit(train_X, train_y, epochs=100, batch_size=64, validation_data=(test_X, test_y))

# 검증 데이터에 대한 예측
test_pred = model.predict(test_X)

# RMSE 계산
rmse = sqrt(mean_squared_error(test_y, test_pred))
print("Root Mean Squared Error: ", rmse)
